!pip install pytorch_lightning

!pip install transformers

In [20]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# %matplotlib inline

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import FunnelTokenizerFast, FunnelModel
transformers.logging.set_verbosity_error()

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

from argparse import ArgumentParser

In [21]:
#파서 만들기
parser = ArgumentParser(description="baseline")
#인자추가하기

#사전학습 모델
parser.add_argument('--text_pretrained_model', default="kosroberta", type=str)
parser.add_argument('--truncation_side', default='right', type=str)
parser.add_argument('--pos_weight', default=None, type=int)
#옵티마이저
parser.add_argument('--optimizer', default="adamw", type=str)
#학습률
parser.add_argument('--learning_rate', default=0.00003, type=float)
parser.add_argument('--scheduler', default="none", type=str)
parser.add_argument('--batch_size', default=64, type=int)
parser.add_argument('--epochs', default=10, type=int)
parser.add_argument('--cv', default=5, type=int)
parser.add_argument('--seed', default=826, type=int)
parser.add_argument('--mixed_precision', default=16, type=int)
parser.add_argument('--device', default=0, type=int)
parser.add_argument('--num_workers', default=0, type=int)
args = parser.parse_args('')

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#각각의 변수는 파서의 인자에서 가져올거다
BATCH_SIZE = args.batch_size
EPOCHS = args.epochs
CV = args.cv
SEED = args.seed

#시드세팅
def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(SEED)

set_seeds()

idx = f"{args.text_pretrained_model}_pw{args.pos_weight}"
idx

Seed set to 826


'kosroberta_pwNone'

- argparse :: 프로그램에 필요한 인자를 사용자 친화적인 명령행 인터페이스로 쉽게 작성하도록 돕는 라이브러리. 
  - 즉, command 창에서 프로그램 내의 인자를 조절하게끔 도와줌

- 흐름은 다음과 같다

##main.py 

import argparse

#파서 만들기</br>
parser = argparse.ArgumentParser(description='Argparse example')

#인자 추가하기</br>
parser.add_argument('--learning_rate', '-lr', type=float, default=0.001, help='learning rate')</br>
parser.add_argument('--mode', type=str, defualt='train', choices=['train', 'test'])</br>
parser.add_argument('--no_gpu', action='store_true')</br>
parser.add_argument('--epochs', type=int, required=True, help='number of epochs')</br>

#내가 쓴 인자 저장</br>
args = parser.parse_args()

#인자 사용</br>
lr = args.lr

In [22]:
if args.text_pretrained_model == "kosroberta":
    text_pretrained_model = "jhgan/ko-sroberta-multitask"
    latent_dim= 384 #* 2
if args.text_pretrained_model == "bert-kor-base":
    text_pretrained_model = "kykim/bert-kor-base"
    latent_dim= 384 #* 2
if args.text_pretrained_model == "funnel-kor-base":
    text_pretrained_model = "kykim/funnel-kor-base"
    latent_dim= 384 #* 2

In [39]:
train_df = pd.read_csv("train.csv").drop(columns=["id"])
test_df = pd.read_csv("test.csv").drop(columns=["id"])

train_df.shape, test_df.shape

((50, 5), (1100, 4))

In [40]:
train_df.head()
#sentence는 각각 리뷰 텍스트고 라벨은 사람이 작성한 리뷰의 라벨이다.

,sentence1,sentence2,sentence3,sentence4,label
0,"직원들 마음에 들지 않는다는 것은 알겠지만, 가지 말아야 할까? 인터넷에서 싸게 살...",직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,직원들 정말 싸가지 없네요 ㅋㅋㅋㅋ 인터넷에서 더 싸게 구입할 수 있다는 걸 알면서...,직원들의 태도가 정말 별로였어요 ㅋㅋㅋㅋ 가볼만한 가게라는 소문을 듣고 인터넷으로 ...,2
1,분위기 최고! 2층 창문이 넓어서 공기가 통하는 느낌이에요. 조명도 멋지고 음료와 ...,분위기가 너무 좋아요! 2층 창문이 넓어서 쾌적한 느낌이에요. 조명도 아름답고 음료...,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,분위기가 너무 좋아요! 2층 창문이 크고 넓어서 탁 트여있는 느낌이에요. 조명도 예...,3
2,"일단, 장사가 잘 되길 바라는 마음에서 별 다섯 개 드립니다. 간도 딱 맞았고, 저...",일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,일단 저는 장사가 잘되기를 바라는 마음에서 별 다섯 개를 주고 싶어요. 맛도 딱 맞...,"먼저, 칭찬과 응원의 의미로 별 다섯 개를 주고 싶습니다. 간도 딱 맞고, 저는 매...",2
3,"1편의 독특함 때문에 살짝 뒤로 밀린 느낌이 있지만, 여전히 재미있어요. 게임 시스...","1편의 신선함에 비해 약간 빛이 바래 보이지만, 여전히 재미있게 즐길 수 있어요. ...","1편의 독특함 때문에 약간의 비교가 불가피하지만, 이 게임은 여전히 흥미로워요. 시...",1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,4
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",빵점을 주고 싶지만 참아냈습니다... 이 영화는 사상 유래가 없는 것 같아요. 조지...,빵점 주고 싶을 정도로 엄청 실망했어요... 이 영화는 별들의 전쟁처럼 역사적인 작...,"빵점을 주고 싶었는데 참았어요... 이런 영화는 전례가 없는데, 조지 루카스의 스타...",1


In [41]:
train_true = []
train_false = []
#리뷰를 사람이 썼는지 기계가 썼는지 0과 1로만 구분한다. 사람이 썼으면 1. 기계가 썼으면0이다
for i, v in enumerate(train_df["label"]):
    for j in range(1, 5):
        if v==j:
            train_true.append(train_df.iat[i, j-1])
        else:
            train_false.append(train_df.iat[i, j-1])
            
train_df_true = pd.DataFrame(train_true, columns=["text"])
train_df_false = pd.DataFrame(train_false, columns=["text"])
train_df_true["label"] = 1
train_df_false["label"] = 0

train_df = pd.concat([train_df_true, train_df_false]).reset_index(drop=True)
train_df.shape

(200, 2)

In [42]:
train_df["label"].value_counts()

0    150
1     50
Name: label, dtype: int64

In [44]:
train_df.head()

,text,label
0,직원들 진짜 싸가지 없어요 ㅋㅋㅋㅋ 가지 마숑 인터넷이 더 싼거 알면서도 이것저것...,1
1,분위기가 짱!! 2층 창문이 커서 탁 트여있는 느낌이에요 ㅎㅎ 조명도 예쁘고 음료랑...,1
2,일단 장사가 잘되길 바라는 마음에서 별5개 드립니다 간도 맞았고 매운걸 좋아하는 입...,1
3,1편이 워낙 참신했던 탓에 좀 묻힌 감이 있긴 하지만 재미는 여전합니다. 시스템도 ...,1
4,"빵점 주고 싶은걸 간신히 참았다...이런건 사상 유래가 없는,조지 루카스 영감의 스...",1


- ID 변수를 기준으로 원래 데이터셋에 있던 여러개의 칼럼 이름을 'variable' 칼럼에 위에서 아래로 길게 쌓아놓고,
- 'value' 칼럼에 ID와 variable에 해당하는 값을 넣어주는 식으로 데이터를 재구조화

In [45]:
test_df = test_df.reset_index()
#데이터 재구조화
test_df = pd.melt(test_df, id_vars='index', var_name="sentence", value_name="text")
test_df.shape

(4400, 3)

In [49]:
test_df.head()

,index,sentence,text
0,0,sentence1,"배송은 정말 빨랐어요! 집에서 속옷이 불편한데 입기 싫고, 안 입으면 민망해서 구매..."
1,1,sentence1,하.. 원래도 비쌋는데 가격도 더ㅠ올라가고 품질은 더 떨어졌어요.. 김밥에 참...
2,2,sentence1,"평일 저녁에 방문했을 때, 서비스가 좋지 않았습니다. 음료를 준비하는 듯 보여 기다..."
3,3,sentence1,"직원들의 친절도가 좋지 않았고, 손님이 오셨음에도 불구하고 무시하는 태도를 보여서 ..."
4,4,sentence1,갈비를 주문했음에도 다른 메뉴가 나왔어요. 그런데 고기의 형태가 기대와 달라 실망스...


In [46]:
X = train_df["text"].values
y = train_df["label"].values
X_test = test_df["text"].values

X.shape, y.shape, X_test.shape

((200,), (200,), (4400,))

## data loader
 #### FunnelTokenizerFast
 - Hugging Face Transformers 라이브러리에서 제공하는 토큰화 도구
 - 아래 질문을 중심으로 모델을 디자인했다.
  - 의미 벡터를 잘 압축해서 더 효과적인 모델을 일반적으로 적용가능하게 만들 수 있을까?
  - 어떻게 사전훈련된 모델의 의미들을 잘 보존할 수 있을까?

In [50]:
class TextDataset(Dataset):
    def __init__(self, sentence, label):
        self.sentence = sentence
        self.label = label
        if args.text_pretrained_model == "funnel-kor-base":
            self.tokenizer = FunnelTokenizerFast.from_pretrained(text_pretrained_model)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(text_pretrained_model)
        if args.truncation_side == "left":
            self.tokenizer.truncation_side = 'left'

    def __len__(self):
        return len(self.sentence)

    def __getitem__(self, idx):
        sentence = self.sentence[idx]

        encoded = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=True,
            return_attention_mask=True,
        )
        
        input_ids = encoded["input_ids"][0]
        token_type_ids = encoded["token_type_ids"][0]
        attention_masks = encoded["attention_mask"][0]
        
        if self.label is not None:
            label = self.label[idx]
            return [input_ids, token_type_ids, attention_masks], label
        else:
            return [input_ids, token_type_ids, attention_masks]

## model

In [58]:
class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x
#모델 생성
class TextModel(nn.Module):
    def __init__(self, latent_dim=latent_dim):
        super().__init__()
        if args.text_pretrained_model == "funnel-kor-base":
            self.txt_model = FunnelModel.from_pretrained(text_pretrained_model)
        else:
            self.txt_model = AutoModel.from_pretrained(text_pretrained_model)
        self.classifier = nn.Sequential(
            SwiGLU(),
            nn.Linear(latent_dim, 1, bias=False)
        )

    def forward(self, x):
        input_ids = x[0]
        token_type_ids = x[1]
        attention_mask = x[2]
        
        txt_side = self.txt_model(
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )
        
        txt_feature = txt_side.last_hidden_state[:, 0, :]
        
        output = self.classifier(txt_feature)
        
        return output

In [59]:
class TextClassifier(pl.LightningModule):
    #문자 분류기
    def __init__(self, backbone, args):
        super().__init__()
        self.backbone = backbone
        if args.pos_weight != None:
            self.pos_weight = torch.tensor(args.pos_weight)

    def forward(self, x):
        predictions = self.backbone(x)
        return predictions

    def step(self, batch):
        x, y = batch
        y_hat = self.backbone(x)
        if args.pos_weight != None:
            loss = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)(y_hat.squeeze(), y.float())
        else:
            loss = nn.BCEWithLogitsLoss()(y_hat.squeeze(), y.float())
        return loss, y, y_hat

    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log('train_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
        self.log("test_acc", acc)

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        y_hat = self.backbone(batch)
        return y_hat

    def configure_optimizers(self):
        if args.optimizer == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=args.learning_rate, momentum=0.9)
        if args.optimizer == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=args.learning_rate)
        if args.optimizer == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=args.learning_rate)
        
        if args.scheduler == "none":
            return optimizer
        if args.scheduler == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer=optimizer,
                step_size=2,
                gamma=0.9,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer=optimizer,
                T_max=EPOCHS,
                eta_min=1e-5,
            )
            return [optimizer], [scheduler]
        if args.scheduler == "onecyclelr":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer=optimizer,
                max_lr=args.learning_rate,
                epochs=EPOCHS,
                steps_per_epoch=int(len(train_index) / BATCH_SIZE),
                pct_start=0.1,
            )
            return [optimizer], [scheduler]

## main

In [60]:
## preprocessing.py

val_ac
c_list = []
preds_list = []

skf = StratifiedKFold(n_splits=CV, shuffle=True, random_state=SEED)

for i, (train_index, val_index) in enumerate(skf.split(X, y)):

    X_train = X[train_index]
    X_val = X[val_index]

    y_train = y[train_index]
    y_val = y[val_index]

## data_loaders.py

    train_ds = TextDataset(X_train, y_train)
    val_ds = TextDataset(X_val, y_val)
    test_ds = TextDataset(X_test, None)

    train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=args.num_workers)
    val_dataloader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)
    test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=args.num_workers)

## train.py

    model = TextClassifier(TextModel(), args)

    callbacks = [
        pl.callbacks.ModelCheckpoint(
            dirpath="saved/", filename=f"{idx}_{i}",
            monitor="val_acc", mode="max"
        ),
    ]
    
    trainer = pl.Trainer(
        max_epochs=EPOCHS, accelerator="auto", callbacks=callbacks,
        precision=args.mixed_precision,
        devices=[args.device]
    )

    trainer.fit(model, train_dataloader, val_dataloader)
    
    ckpt = torch.load(f"saved/{idx}_{i}.ckpt")
    model.load_state_dict(ckpt['state_dict'])

## test.py

    eval_dict = trainer.validate(model, dataloaders=val_dataloader)[0]
    val_acc_list.append(eval_dict["val_acc"])
    
    y_preds = trainer.predict(model, dataloaders=test_dataloader)
    preds_list.append(np.vstack(y_preds))
    
val_acc_mean = np.mean(val_acc_list)

print(f"val_acc_mean: {val_acc_mean}")

AttributeError: module 'torch.backends' has no attribute 'mps'